<a href="https://colab.research.google.com/github/pterodactylus42/textReGenerator/blob/main/lstm_mit_embedding_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests

url = 'https://www.gutenberg.org/cache/epub/38780/pg38780.txt'
response = requests.get(url)

filename = 'etah-murr.txt'

with open(filename, 'wb') as file:
    file.write(response.content)

print('Download complete!')

Download complete!


In [3]:
with open("etah-murr.txt", "r", encoding="utf-8") as file:
    contents = file.read()

contents = contents.split("\n")
contents = [line.strip() for line in contents if "#" not in line]

contents = "\n".join(contents)

In [4]:
!pip install nltk scikit-learn # do this in correct env if running local

In [5]:
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')
nltk.download('punkt_tab')

tokens = word_tokenize(contents)

# use max_features=len(set(tokens)) for full coverage, if you have too much ram
# use max_features=1000 for less coverage, if "Your session crashed after using all available RAM"
cv = CountVectorizer(max_features=len(set(tokens)), lowercase=False, token_pattern="(.*)")
cv.fit(tokens)

features = cv.get_feature_names_out()

word_to_int = {}
int_to_word = {}

for i in range(0, len(features)):
    word = features[i]

    word_to_int[word] = i
    int_to_word[i] = word

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [6]:
tokens_transformed = [word_to_int[word] for word in tokens if word in word_to_int]

In [7]:
import numpy as np

X = []
y = []

seq_length = 40

for i in range(0, len(tokens_transformed) - seq_length):
    X.append(tokens_transformed[i:i+seq_length])
    y.append(tokens_transformed[i+seq_length])

X = np.array(X)
y = np.array(y)

In [10]:
!pip install keras tensorflow # in correct env

In [11]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding, Flatten

model = Sequential()
model.add(Embedding(cv.max_features, 150, input_shape=(seq_length,)))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=False))

model.add(Dense(1000, activation="sigmoid"))

model.add(Dense(cv.max_features, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("etah.{epoch:02d}-{val_loss:.2f}.keras")

model.fit(
    X,
    to_categorical(y, num_classes=cv.max_features),
    epochs=40,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint])

In [ ]:
# model.summary()

In [ ]:
model.save("etah.keras")

In [ ]:
import pickle

with open("word_to_int.etah.pickle", "wb") as file:
    pickle.dump(word_to_int, file)


with open("int_to_word.etah.pickle", "wb") as file:
    pickle.dump(int_to_word, file)